# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import pickle
import numpy as np
from sklearn.externals import joblib
from sklearn import *
from sklearn import cluster
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing, cross_validation, neighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\madhu\Anacon

## Load clean loan csv 

In [3]:
loan_df = pd.read_csv('../Loan.csv', low_memory=False) 

In [4]:
loan_df.columns

Index(['id', 'debt_settlement_flag', 'application_type', 'fico_range_low',
       'fico_range_high', 'emp_length', 'dti', 'annual_inc', 'grade',
       'sub_grade', 'int_rate', 'loan_amnt', 'issue_d', 'purpose', 'State',
       'home_ownership', 'zip_code', 'policy_code', 'term', 'Year', 'Month',
       'fico', 'approval'],
      dtype='object')

# Remove General Columns Not Needed

In [5]:
feature_df = loan_df[['application_type','emp_length', 'dti', 'annual_inc',
       'int_rate', 'loan_amnt', 'purpose', 'State', 'home_ownership', 'term', 'fico', 'approval']]

In [6]:
home_ownerships = pd.get_dummies(feature_df['home_ownership'], prefix='house')
feature_df = feature_df.join(home_ownerships)
feature_df.drop('home_ownership', axis=1, inplace=True)

app_type = pd.get_dummies(feature_df['application_type'], prefix='appType')
feature_df = feature_df.join(app_type)
feature_df.drop('application_type', axis=1, inplace=True)

adr_states = pd.get_dummies(feature_df['State'], prefix='s')
feature_df = feature_df.join(adr_states)
feature_df.drop('State', axis=1, inplace=True)

#purpose   
purpose = pd.get_dummies(feature_df['purpose'], prefix='purpose')
feature_df = feature_df.join(purpose)
feature_df.drop('purpose', axis=1, inplace=True)

# No Clustering

In [10]:
zero_cluster_df = feature_df

In [11]:
X_train, X_test, y_train, y_test = train_test_split(zero_cluster_df.ix[:, zero_cluster_df.columns != 'int_rate'], 
                                            zero_cluster_df['int_rate'], 
                                            test_size=0.30)

In [12]:
# Creating the model
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [25]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.42777569429
-----Train-----
RMS:  3.402651762736587
MAE:  2.62685705323
MAPE:  19.497676313563026
-----Test-----
RMS:  4.17946334159284
MAE:  3.22313535767
MAPE:  23.90864436620862


# Manual Clustering

In [14]:
manual_df = feature_df
def clustering(fico):
    cluster_name = ''
    if fico>790.0:
        cluster_name ='K1'
    elif ((fico <= 790.0) & (fico>750.0)):
        cluster_name = 'K2'
    elif ((fico <=750.0) & (fico>700.0)):
        cluster_name ='K3'
    elif ((fico <=700.0) & (fico>660.0)):
        cluster_name= 'K4'
    return cluster_name
manual_df['cluster'] = manual_df['fico'].astype(float).map(lambda x: clustering(x))

In [15]:
manual_df.head()

,emp_length,dti,annual_inc,int_rate,loan_amnt,term,fico,approval,house_ANY,house_MORTGAGE,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,cluster
0,10,27.65,24000.0,10.65,5000.0,36,737.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
1,1,1.00,30000.0,15.27,2500.0,60,742.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
2,10,8.72,12252.0,15.96,2400.0,36,737.0,1,0,0,...,0,0,0,0,0,0,1,0,0,K3
3,10,20.00,49200.0,13.49,10000.0,36,692.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4
4,1,17.94,80000.0,12.69,3000.0,60,697.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4


In [16]:
#Seperate into diff dataframes
manual_c1 = manual_df[manual_df['cluster']=='K1']
manual_c1.drop('cluster',axis=1,inplace=True)
manual_c2 = manual_df[manual_df['cluster']=='K2']
manual_c2.drop('cluster',axis=1,inplace=True)
manual_c3 = manual_df[manual_df['cluster']=='K3']
manual_c3.drop('cluster',axis=1,inplace=True)
manual_c4 = manual_df[manual_df['cluster']=='K4']
manual_c4.drop('cluster',axis=1,inplace=True)

#### Manual 1

In [17]:

X_train, X_test, y_train, y_test = train_test_split(manual_c1.ix[:, manual_c1.columns != 'int_rate'], 
                                            manual_c1['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [18]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.423278086141
-----Train-----
RMS:  2.4132534196765483
MAE:  1.69512610653
MAPE:  20.174913249863568
-----Test-----
RMS:  2.927408037014591
MAE:  2.06371910775
MAPE:  24.790777898585972


#### Manual 2

In [19]:
#manual 2
X_train, X_test, y_train, y_test = train_test_split(manual_c2.ix[:, manual_c2.columns != 'int_rate'], 
                                            manual_c2['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [20]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.408037804256
-----Train-----
RMS:  2.797397310813602
MAE:  2.02220727206
MAPE:  22.139867143314593
-----Test-----
RMS:  3.4388155022276075
MAE:  2.48492245828
MAPE:  27.211092286203748


#### Manual 3

In [21]:
#Manual 3
X_train, X_test, y_train, y_test = train_test_split(manual_c3.ix[:, manual_c3.columns != 'int_rate'], 
                                            manual_c3['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [22]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.442430657324
-----Train-----
RMS:  3.18970394615286
MAE:  2.43349092967
MAPE:  22.311900727780014
-----Test-----
RMS:  3.9203478517705777
MAE:  2.99003039134
MAPE:  27.398009627958736


#### Manual 4

In [23]:
#Manual 4
X_train, X_test, y_train, y_test = train_test_split(manual_c4.ix[:, manual_c4.columns != 'int_rate'], 
                                            manual_c4['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [24]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.42777569429
-----Train-----
RMS:  3.402651762736587
MAE:  2.62685705323
MAPE:  19.497676313563026
-----Test-----
RMS:  4.17946334159284
MAE:  3.22313535767
MAPE:  23.90864436620862


# K-Means

In [7]:
k_cluster = joblib.load(open('../kmeanCluster.pkl', 'rb'))
#Create new column in load_df for the k-means clustering 
k_mean_feature_df = feature_df
k_mean_feature_df['kMean'] = k_cluster.labels_

In [8]:
kmean_0 = k_mean_feature_df[k_mean_feature_df['kMean']==0]
kmean_0.drop('kMean', axis=1, inplace=True)
kmean_1 = k_mean_feature_df[k_mean_feature_df['kMean']==1]
kmean_1.drop('kMean', axis=1, inplace=True)
kmean_2 = k_mean_feature_df[k_mean_feature_df['kMean']==2]
kmean_2.drop('kMean', axis=1, inplace=True)
kmean_3 = k_mean_feature_df[k_mean_feature_df['kMean']==3]
kmean_3.drop('kMean', axis=1, inplace=True)
kmean_4 = k_mean_feature_df[k_mean_feature_df['kMean']==4]
kmean_4.drop('kMean', axis=1, inplace=True)
kmean_5 = k_mean_feature_df[k_mean_feature_df['kMean']==5]
kmean_5.drop('kMean', axis=1, inplace=True)
kmean_6 = k_mean_feature_df[k_mean_feature_df['kMean']==6]
kmean_6.drop('kMean', axis=1, inplace=True)
kmean_7 = k_mean_feature_df[k_mean_feature_df['kMean']==7]
kmean_7.drop('kMean', axis=1, inplace=True)


#### K-means0

In [9]:
#K-mean_0
X_train, X_test, y_train, y_test = train_test_split(kmean_0.ix[:, kmean_0.columns != 'int_rate'], 
                                            kmean_0['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [10]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.465213648413
-----Train-----
RMS:  3.165509290934963
MAE:  2.41715560266
MAPE:  21.109717818546713
-----Test-----
RMS:  3.8819652623172947
MAE:  2.96543612717
MAPE:  25.908108949460836


#### K-means1

In [11]:
#K-mean_1
X_train, X_test, y_train, y_test = train_test_split(kmean_1.ix[:, kmean_1.columns != 'int_rate'], 
                                            kmean_1['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)# Accuracy of the model created

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [12]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.483736198004
-----Train-----
RMS:  3.5459357066693404
MAE:  2.73329479989
MAPE:  19.768568426856
-----Test-----
RMS:  4.377867964260734
MAE:  3.38174637409
MAPE:  24.54556532824701


#### K-means2

In [14]:
#K-mean_2
X_train, X_test, y_train, y_test = train_test_split(kmean_2.ix[:, kmean_2.columns != 'int_rate'], 
                                            kmean_2['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [15]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.398955417951
-----Train-----
RMS:  3.273660625364069
MAE:  2.52059155087
MAPE:  22.78728295993902
-----Test-----
RMS:  4.030707275309413
MAE:  3.10175868158
MAPE:  28.167610473011152


#### K-means3

In [16]:
#K-mean_3
X_train, X_test, y_train, y_test = train_test_split(kmean_3.ix[:, kmean_3.columns != 'int_rate'], 
                                            kmean_3['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [17]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.476409426604
-----Train-----
RMS:  3.2866141748821276
MAE:  2.51411422467
MAPE:  20.79735756992456
-----Test-----
RMS:  4.04429627832292
MAE:  3.08724002393
MAPE:  25.623135924124036


#### K-means4

In [18]:
#K-mean_4
X_train, X_test, y_train, y_test = train_test_split(kmean_4.ix[:, kmean_4.columns != 'int_rate'], 
                                            kmean_4['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [19]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.287108450056
-----Train-----
RMS:  4.545174449293427
MAE:  3.47098389356
MAPE:  24.346854326285257
-----Test-----
RMS:  5.481590773832419
MAE:  4.21031699346
MAPE:  29.94115542437242


#### K-means5

In [20]:
#K-mean_5
X_train, X_test, y_train, y_test = train_test_split(kmean_5.ix[:, kmean_5.columns != 'int_rate'], 
                                            kmean_5['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [21]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.476384458812
-----Train-----
RMS:  3.4931695465666337
MAE:  2.68951516863
MAPE:  20.611693470936636
-----Test-----
RMS:  4.319963405715326
MAE:  3.32001325958
MAPE:  25.52807435453498


#### K-mean_6

In [22]:
#K-mean_6
X_train, X_test, y_train, y_test = train_test_split(kmean_6.ix[:, kmean_6.columns != 'int_rate'], 
                                            kmean_6['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [23]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.447892926258
-----Train-----
RMS:  3.6370185750340984
MAE:  2.80419921662
MAPE:  19.499297632251476
-----Test-----
RMS:  4.428955396978004
MAE:  3.41809492534
MAPE:  24.00364040096457


#### K-means7

In [24]:
#K-mean_7
X_train, X_test, y_train, y_test = train_test_split(kmean_7.ix[:, kmean_7.columns != 'int_rate'], 
                                            kmean_7['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [25]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.442204985408
-----Train-----
RMS:  3.111921898167613
MAE:  2.38280574456
MAPE:  21.66866250096642
-----Test-----
RMS:  3.795561575541108
MAE:  2.91327672242
MAPE:  26.700319709216767
